In [4]:
%cd /content/drive/MyDrive/yolo_workspace/yolov5

/content/drive/MyDrive/yolo_workspace/yolov5


In [5]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

Cloning into 'yolov5'...
remote: Enumerating objects: 13025, done.
remote: Counting objects: 100% (200/200), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 13025 (delta 123), reused 177 (delta 108), pack-reused 12825
Receiving objects: 100% (13025/13025), 12.46 MiB | 6.93 MiB/s, done.
Resolving deltas: 100% (8949/8949), done.
/content/drive/MyDrive/yolo_workspace/yolov5/yolov5
HEAD is now at fbe67e4 Fix `OMP_NUM_THREADS=1` for macOS (#8624)


In [ ]:
!curl -L "https://public.roboflow.com/ds/xvVyXDIOCY?key=aEq3bwix64" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [7]:
# define number of classes based on YAML
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [8]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [9]:
%%writetemplate /content/drive/MyDrive/yolo_workspace/yolov5/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [1]:
%cd /content/drive/MyDrive/yolo_workspace/yolov5/yolov5

/content/drive/MyDrive/yolo_workspace/yolov5/yolov5


In [2]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

     |████████████████████████████████| 596 kB 19.5 MB/s 
Setup complete. Using torch 1.12.0+cu113 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [3]:
!python val.py --weights runs/best.pt --data data_test.yaml --img 416 --iou-thres 0.5 --task speed --device 0

val: data=data_test.yaml, weights=['runs/best.pt'], batch_size=32, imgsz=416, conf_thres=0.001, iou_thres=0.5, task=speed, device=0, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
custom_YOLOv5s summary: 232 layers, 7251912 parameters, 0 gradients, 16.8 GFLOPs
100% 755k/755k [00:00<00:00, 119MB/s]
val: Scanning '/content/drive/MyDrive/yolo_workspace/yolov5/yolov5/test/labels.cache' images and labels... 36 found, 0 missing, 0 empty, 0 corrupt: 100% 36/36 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 2/2 [00:39<00:00, 19.69s/it]
                 all         36        471      0.785      0.867      0.854      0.585
           Platelets         36         36      0.747      0.8

In [4]:
!python val.py --weights runs/best.pt --data data_test.yaml --img 416 --iou-thres 0.5 --task study --device 0

val: data=data_test.yaml, weights=['runs/best.pt'], batch_size=32, imgsz=416, conf_thres=0.001, iou_thres=0.5, task=study, device=0, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False

Running study_data_test_best.txt --imgsz 256...
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
custom_YOLOv5s summary: 232 layers, 7251912 parameters, 0 gradients, 16.8 GFLOPs
val: Scanning '/content/drive/MyDrive/yolo_workspace/yolov5/yolov5/test/labels.cache' images and labels... 36 found, 0 missing, 0 empty, 0 corrupt: 100% 36/36 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 2/2 [00:00<00:00,  2.24it/s]
                 all         36        471      0.844      0.854      0.898      0.564
           Platelets         36         36      0.7

In [6]:
!python detect.py --weights runs/best.pt --img 416 --conf 0.4 --source test/images --device 0

detect: weights=['runs/best.pt'], source=test/images, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
custom_YOLOv5s summary: 232 layers, 7251912 parameters, 0 gradients, 16.8 GFLOPs
image 1/36 /content/drive/MyDrive/yolo_workspace/yolov5/yolov5/test/images/BloodImage_00038_jpg.rf.ffa23e4b5b55b523367f332af726eae8.jpg: 416x416 2 Plateletss, 23 RBCs, 1 WBC, Done. (0.010s)
image 2/36 /content/drive/MyDrive/yolo_workspace/yolov5/yolov5/test/images/BloodImage_00044_jpg.rf.e7760375eba4bc20c5746367e2311e18.jpg: 416x416 3 Plateletss, 19 RBCs, 2 WBCs, Done. (0.01